<a href="https://colab.research.google.com/github/VivekanandaMudelli/hand_drawn_sketch_recognition/blob/main/logstic_regression(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# file_id = "1EFQlS9C4YGT5CNjD_NX_Q-HaiyLH4sjE"
file_id = "14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q"

# Construct the direct download URL
# https://drive.google.com/file/d/14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q/view?usp=sharing
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output = "data.csv"  # Change filename as needed
gdown.download(download_url, output, quiet=False)

# Read the CSV file
data = pd.read_csv(output)
data = data.dropna()

# X = data.drop(columns = ['label'])
# y = data['label']
X = data.drop(data.columns[0],axis = 1).drop(columns = ['encoded_part','extracted_part'])
y = data['extracted_part']

print(X.shape)
print(y.shape)

Downloading...
From (original): https://drive.google.com/uc?id=14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q
From (redirected): https://drive.google.com/uc?id=14X_8QdPW12hGHnNzZztuz3keHOo9Cq4q&confirm=t&uuid=91d44b2e-f47a-44c1-9690-dc36e094b388
To: /content/data.csv
100%|██████████| 793M/793M [00:09<00:00, 82.9MB/s]


In [ ]:
# Mean
X_mean = X.mean()

# Standard deviation
X_std = X.std()

# Standardization
Z = (X - X_mean) / X_std.replace(0,1e-10)

In [ ]:
# Importing PCA
from sklearn.decomposition import PCA

# Taking top 50 dimensions
pca = PCA(n_components=500)
pca.fit(Z)
x_pca = pca.transform(Z)

# Create the dataframe
df = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(500)])
print(df)

In [ ]:
df = X

In [ ]:
df['label'] = y

In [ ]:
df

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm # Import tqdm

In [ ]:
# Assuming your DataFrame is named 'df' and the target variable is 'label'
X = df.drop(columns=['label']).values  # Features as NumPy array
y = df['label'].values  # Target variable as NumPy array
#Encode the target variable if it contains strings
le = LabelEncoder()  # Create a LabelEncoder object
y = le.fit_transform(y)  # Transform the target variable
# Get the number of unique classes
num_classes = len(np.unique(y))
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Add bias term to X
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

In [ ]:
# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# Logistic Regression training function (same as before)
def train_logistic_regression(X, y, lr=0.01, epochs=1000):
    m, n = X.shape
    theta = np.zeros(n)  # Initialize weights
    for epoch in range(epochs):
        z = X @ theta
        h = sigmoid(z)
        gradient = (X.T @ (h - y)) / m
        theta -= lr * gradient
        if epoch % 100 == 0:  # Print loss every 100 epochs
            loss = -np.mean(y * np.log(h + 1e-9) + (1 - y) * np.log(1 - h + 1e-9))
            print(f"Epoch {epoch}, Loss: {loss}")
    return theta

In [ ]:
#One-vs-Rest Training:
all_theta = []  # To store the parameters for each class
for i in tqdm(range(num_classes), desc="Training"):  # Wrap range with tqdm
    # Create a binary target variable for the current class
    y_train_binary = (y_train == i).astype(int)
    # Train a logistic regression model for this class
    theta = train_logistic_regression(X_train_bias, y_train_binary, lr=0.1, epochs=1000)
    all_theta.append(theta)

In [ ]:
#Prediction
y_pred_probs = []
for i in range(num_classes):
    z = X_test_bias @ all_theta[i]
    y_pred_probs.append(sigmoid(z))
# Get the class with the highest probability
y_pred = np.argmax(y_pred_probs, axis=0)
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")